# Numpy进阶

Numpy 的数组对象是用于存储大数据集的结构类型实例，数组对象是同质、可变、可重复、有序、无映射的数据集：

| 结构      | 同质 | 可变 | 重复 |  有序  | 映射 |
|------------:|-----:|-----:|-----:|-----:|-----:|
|列表（list） | 否   |  是 | 是   | 是 |  否  |
|数组（ndarray）| 是   | 是 | 是   | 是 |  否  |

本章将深入深入介绍数组对象更多细节，了解数组视图与深拷贝等概念，了解数组运算过程中的广播机制。介绍Numpy 中常用的随机数子包`np.random`、多项式拟合等功能函数，并通过一些实例来介绍 Numpy 的应用。

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 视图与深拷贝

前面介绍过，数组对象的数据结构如图所示：
![ndarray数据结构](../images/numpy_ndarray_type.png)

数组的数据实际存放在对象属性`ndarray.data`所指对象，通过数组位置索引获取元素数据。Numpy 数组通常用于存储大数据集，数组对象的各种运算操作都会使用这些数据，数组数据的拷贝需要花费较多时间，故而数组对象的操作并非都需要数据拷贝。根据数组数据是否拷贝，可把数组对象的操作分为以下三种情况：
- 完全不拷贝
- 视图或浅拷贝
- 深拷贝

### 完全不拷贝

Python 的赋值操作会创建一个新的变量，然后把变量绑定到一个对象。故而，通过赋值操作来创建一个变量，并指向已有数组对象，这种赋值操作不会拷贝任何数据：

In [20]:
arr1 = np.array(range(1024))
arr2 = arr1
arr2 is arr1

True

### 视图或浅拷贝

在 Numpy 中，数组对象的视图又称为浅拷贝，会创建一个新的数组对象，但会与原来的数组公用同一份数据。如下图所示：
![Numpy 视图示意](../images/numpy_view_data.png)

例如，下面使用切分创建新的数组对象：

In [28]:
arr1 = np.array(range(1024))
arr2 = arr1[256:512]

可以使用内置函数`id()`来查看两个数组对象的身份标识，或使用身份运算符比较：

In [29]:
print(id(arr1), id(arr2), arr1 is arr2)

1842153120112 1842153120272 False


由上可知，变量`arr1`与`arr2`指向两个数组对象。数组对象使用属性`ndarray.flags.owndata`来标识其数据是自己数据或是视图：

In [30]:
print(arr1.flags.owndata, arr2.flags.owndata)

True False


可以看出数组对象`arr1`的数据属于自身，而数组对象`arr1`则是视图。还可以通过数组对象属性`base`来检查数组视图数据来自哪个数组：

In [31]:
# 数组不是视图
print(arr1.base is None)
# arr2 是 arr1 的视图
print(arr2.base is arr1)

True
True


如果两个数组对象使用共享同一个数据，当其中更改数组中的数据，那么两个数组对应的数据都会得到更改：

In [37]:
# 原值
print(arr1[256], arr2[0], arr1[257], arr2[1])
# 更改
arr2[0] = 1
arr1[257] = 2 
print(arr1[256], arr2[0], arr1[257], arr2[1])

1 1 2 2
1 1 2 2


### 深拷贝

有时候需要完全拷贝数组对象来创建全新的数组对象。数组对象提供有方法`ndarray.copy()`来实现：

In [42]:
arr3 = arr1.copy()
print(arr1.base is None, arr3.base is None)

True True


此种情况下，更改数组中的元素，则互不影响：

In [43]:
print(arr1[256], arr3[256])
arr3[256] = 0
print(arr1[256], arr3[256])

1 1
1 0


通过索引数组和布尔数组获取的数组对象都是深拷贝，也称之为`Fancy Indexing`。

In [45]:
arr4 = arr1[arr1 % 10 == 0]
print(arr1.base is None, arr4.base is None)

True True


在使用 Numpy 过程中，需要注意浅拷贝与深拷贝的区别。

## 广播（broadcasting）

数组一个好处是矢量化运算，即两个数组之间的运算支持元素级（element-wise）运行，直接进行数组的运算而不用写循环代码。

In [47]:
arr1 = np.array([1.0, 2.0, 3.0, 4.0])
arr2 = np.array([-1.0, -2.0, -3.0, -4.0])
arr1 + arr2

array([0., 0., 0., 0.])

当数组的维度不相同时，如何进行运算，会不会出现异常？

Numpy 中的数组运算支持广播（broadcasting）。广播是不同形状的数组进行算术运算方式。Numpy中广播的规则如下：
1. 如果数组的维数不等，则都向维数最多的数组看齐，维数较小的数组通过在前面加1补齐，直到维数相等；
2. 依次比较两个数组每个维度的大小，当数组在某个维度长度为1，则可以沿这个维度进行扩充，直到维度大小相同。

也就是说，两个数组的尾部维度（trailing dimension）的轴长度一致或者一方长度为1，则可以进行广播。广播会在缺失维度或长度为1的维度上进行。

下面看一个最简单的广播方式，即数组与标量值之间的计算：

In [48]:
arr1 * np.pi

array([ 3.14159265,  6.28318531,  9.42477796, 12.56637061])

这里广播的功能，就是把圆周率标量值转换为与数组 `arr1` 维度一样的数组，再进行乘法。

对于二维的情况，广播具有类似功能。例如一个二维数组

In [53]:
arr2 = np.array([[1.0, 2.0, 3.0], [3.0, 4.0, 5.0], [5.0, 6.0, 7.0], [7.0, 8.0, 9.0]])
arr2

array([[1., 2., 3.],
       [3., 4., 5.],
       [5., 6., 7.],
       [7., 8., 9.]])

In [54]:
# 计算平均值
arr2.mean(axis=0)

array([4., 5., 6.])

In [55]:
demeaned = arr2 - arr2.mean(axis=0)
demeaned

array([[-3., -3., -3.],
       [-1., -1., -1.],
       [ 1.,  1.,  1.],
       [ 3.,  3.,  3.]])

这里广播的作用就是把低纬度的值扩展到高纬度上去。

广播的功能强大，但难于理解，容易出错。使用时要注意。